## 26. Remove Duplicates from Sorted Array

In [ ]:
(defun removeduplicates (nums)
  (length (remove-duplicates nums)))

## 27. Remove Element

In [ ]:
;;; 使用 remove 或 delete 都可以做到，但因为不能使用额外的空间，
;;; 那么可以有副作用的 delete，但原数组最终结果是未知的，不可行。
;;;
;;; 使用 car 和 pop，pop 是原地修改，但在函数中却不行。
;;;
;;; setf 或 rotatef 宏可以做到，最终使用双指针。
(defun remove-element (nums val)
  (let ((n (length nums))
        (i 0))
    (loop while (< i n) do
         (if (= (nth i nums) val)
             (rotatef (nth i nums)
                      (nth (decf n) nums))
             (incf i)))
    n))

## 28. Implement strStr()

In [ ]:
(defun strstr (haystack needle)
  (let ((needle-size (length needle)))
    (loop for i to (- (length haystack)
                      needle-size) do
         (if (string= (subseq haystack i (+ i needle-size))
                      needle)
             (return-from strstr i)))
    -1))

## 29. Divide Two Integers

In [ ]:
;;; 不能用乘、除、求模运算符。
;;; 只用加减的话，时间复杂度与商的大小成正比。
;;; 但位移可以实现类似乘除的操作。
(defun divide (dividend divisor)
  (let ((positive (and (> dividend 0)
                       (> divisor 0)))
        (dividend (abs dividend))
        (divisor (abs divisor))
        (res 0))
    (loop while (> dividend divisor) do
         (let ((temp divisor)
               (m 1))
           (loop while (> dividend temp) do
                (decf dividend temp)
                (incf res m)
                (setf temp (ash temp 1))
                (setf m (ash m 1)))))
    (if (not positive)
        (setf res (- res)))
    (min (max res (- (expt 2 31))) (1- (expt 2 31)))))

## 30. Substring with Concatenation of All Words

In [ ]:
(defun find-substring (s words)
  (let ((word-length (length (car words)))
        (words-size (length words))
        (hash-table (make-hash-table :test 'equal))
        (res))
    ;; 使用 hash-table 计算 words 中每个字符串出现的次数
    (loop for word in words do
         (incf (gethash word hash-table 0)))
    (loop for i to (- (length s) (* word-length words-size)) do
         (let ((plist)) ; 使用属性表存储子串出现次数
           (loop for j below words-size do
                (let* ((start (+ i (* j word-length)))
                       (end (+ start word-length))
                       (substr (subseq s start end))
                       (key (intern substr "KEYWORD")))
                  (if (< (getf plist key 0)
                         (gethash substr hash-table -1))
                      (incf (getf plist key 0))
                      (return)))
              finally (push i res))))
    res))

## 31. Next Permutation

算法参考维基百科 - [全排列生成算法 - 字典序法](https://zh.wikipedia.org/wiki/%E5%85%A8%E6%8E%92%E5%88%97%E7%94%9F%E6%88%90%E7%AE%97%E6%B3%95#%E5%AD%97%E5%85%B8%E5%BA%8F%E6%B3%95)

设P是集合{1，2，……n-1，n}的一个全排列：P=P1P2……Pj-1PjPj+1……Pn（1≤P1，P2，……，Pn≤n-1）
1. 从排列的右端开始，找出第一个比右边数字小的数字的序号j，即j=max{i|Pi<Pi+1，i>j}
2. 在Pj的右边的数字中，找出所有比Pj大的数字中最小的数字Pk，即k=min{i|Pi>Pj，i>j}
3. 交换Pi，Pk
4. 再将排列右端的递减部分Pj+1Pj+2……Pn倒转，因为j右端的数字是降序，所以只需要其左边和右边的交换，直到中间，因此可以得到一个新的排列P'=P1P2……Pj-1PkPn……Pj+2Pj+1。

另可参考 [Next lexicographical permutation algorithm](https://www.nayuki.io/page/next-lexicographical-permutation-algorithm)

In [ ]:
;;; 下一个全排列，in-place
(defun next-permutation (nums)
  (let* ((size (length nums))
         ;; 从右端开始找出算法中的 j
         (j (loop for i from (- size 2) downto 0 do
                 (if (< (nth i nums)
                        (nth (1+ i) nums))
                     (return i)))))
    (if j
        ;; 找到 j 之后寻找算法中的 k
        (let* ((k (1+ j)))
          (loop for i from (1+ j) below size do
               (if (and (< (nth j nums)
                           (nth i nums))
                        (< (nth i nums)
                           (nth k nums)))
                   (setf k i)))
          (when (< (nth j nums)
                   (nth k nums))
            (rotatef (nth j nums) (nth k nums))
            (let ((start (1+ j))
                  (end (1- size)))
              (loop while (< start end) do
                   (rotatef (nth start nums) (nth end nums))
                   (incf start)
                   (decf end)))))
        ;; 未找到符合条件的 j，将输入逆序
        (let ((start 0)
              (end (1- size)))
          (loop while (< start end) do
               (rotatef (nth start nums) (nth end nums))
               (incf start)
               (decf end))))))

## 32. Longest Valid Parentheses

In [ ]:
;;; 刚开始想到用动态规划，后来发现直接使用栈就可以了，更简单清晰。
;;; Lisp 中的列表来代替实现栈，更加方便。
(defun longest-valid-parentheses (s)
  (let ((stack)
        (longest 0))
    (loop for c across s 
       for i to (length s) do
         (if (and (equal (car (car stack)) #\()
                  (equal c #\)))
             (progn
               (pop stack)
               (if (null stack)
                   (setf longest (1+ i))
                   (setf longest (- i (cdr (car stack))))))
             (push (cons c i) stack)))
    longest))

## 33. Search in Rotated Sorted Array

算法参考 [Clever idea making it simple](https://leetcode.com/problems/search-in-rotated-sorted-array/discuss/14435/Clever-idea-making-it-simple) 与 [Pretty short C++/Java/Ruby/Python](https://leetcode.com/problems/search-in-rotated-sorted-array/discuss/14419/Pretty-short-C%2B%2BJavaRubyPython)

假设查找的范围为 `lo` ~ `hi`，`mid` 为范围中间的索引。使 `target` 在 `mid` 右边（`lo = mid + `）的所有可能为：

1. nums[0] > target and (nums[0] > nums[mid] == target > nums[mid])
2. nums[0] < target and nums[0] > nums[mid] and target > nums[mid]

In [ ]:
;;; 
(defun search-rotated-sorted (nums target)
  (let ((lo 0)
        (hi (length nums)))
    (loop while (< lo hi) do
         (let ((mid (floor (+ lo hi) 2)))
           ;; 实现类似异或的操作，`logxor` 使用数值为参数，不符合要求。
           ;; 可使用 `reduce` 宏自己写一个作用于 `t` 与 `nil` 之上的异或操作
           ;; (reduce #'(lambda (x y) (not (equal x y))) sequence)
           (if (not (equal (not (equal (> (first nums) target)
                                       (> (first nums) (nth mid nums))))
                           (> target (nth mid nums))))
               (setf lo (1+ mid))
               (if (= (nth mid nums) target)
                   (return mid)
                   (setf hi mid))))
       finally (return -1))))